# ಪಠ್ಯ ವರ್ಗೀಕರಣ ಕಾರ್ಯ

ಈ ಘಟಕದಲ್ಲಿ, ನಾವು ಸರಳ ಪಠ್ಯ ವರ್ಗೀಕರಣ ಕಾರ್ಯದಿಂದ ಪ್ರಾರಂಭಿಸುವೆವು, ಇದು **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)** ಡೇಟಾಸೆಟ್ ಆಧಾರಿತವಾಗಿದೆ: ನಾವು ಸುದ್ದಿಯ ಶೀರ್ಷಿಕೆಗಳನ್ನು 4 ವರ್ಗಗಳಲ್ಲಿ ಒಂದಾಗಿ ವರ್ಗೀಕರಿಸುವೆವು: ವಿಶ್ವ, ಕ್ರೀಡೆ, ವ್ಯವಹಾರ ಮತ್ತು ವಿಜ್ಞಾನ/ತಂತ್ರಜ್ಞಾನ.

## ಡೇಟಾಸೆಟ್

ಡೇಟಾಸೆಟ್ ಅನ್ನು ಲೋಡ್ ಮಾಡಲು, ನಾವು **[TensorFlow Datasets](https://www.tensorflow.org/datasets)** API ಅನ್ನು ಬಳಸುವೆವು.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

ನಾವು ಈಗ `dataset['train']` ಮತ್ತು `dataset['test']` ಅನ್ನು ಕ್ರಮವಾಗಿ ಬಳಸಿ ಡೇಟಾಸೆಟ್‌ನ ತರಬೇತಿ ಮತ್ತು ಪರೀಕ್ಷಾ ಭಾಗಗಳನ್ನು ಪ್ರವೇಶಿಸಬಹುದು:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


ನಮ್ಮ ಡೇಟಾಸೆಟ್‌ನ ಮೊದಲ 10 ಹೊಸ ಶೀರ್ಷಿಕೆಗಳನ್ನು ಮುದ್ರಿಸೋಣ:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## ಪಠ್ಯ ವೆಕ್ಟರೀಕರಣ

ಈಗ ನಮಗೆ ಪಠ್ಯವನ್ನು ಟೆನ್ಸರ್‌ಗಳಾಗಿ ಪ್ರತಿನಿಧಿಸಬಹುದಾದ **ಸಂಖ್ಯೆಗಳಾಗಿ** ಪರಿವರ್ತಿಸಲು ಅಗತ್ಯವಿದೆ. ನಾವು ಪದಮಟ್ಟದ ಪ್ರತಿನಿಧಾನವನ್ನು ಬಯಸಿದರೆ, ನಾವು ಎರಡು ಕೆಲಸಗಳನ್ನು ಮಾಡಬೇಕಾಗುತ್ತದೆ:

* ಪಠ್ಯವನ್ನು **ಟೋಕನ್‌ಗಳಾಗಿ** ವಿಭಜಿಸಲು **ಟೋಕನೈಜರ್** ಅನ್ನು ಬಳಸಬೇಕು.
* ಆ ಟೋಕನ್‌ಗಳ **ಶಬ್ದಕೋಶ**ವನ್ನು ನಿರ್ಮಿಸಬೇಕು.

### ಶಬ್ದಕೋಶದ ಗಾತ್ರವನ್ನು ಮಿತಿಗೊಳಿಸುವುದು

AG News ಡೇಟಾಸೆಟ್ ಉದಾಹರಣೆಯಲ್ಲಿ, ಶಬ್ದಕೋಶದ ಗಾತ್ರ ಬಹಳ ದೊಡ್ಡದು, 100k ಕ್ಕಿಂತ ಹೆಚ್ಚು ಪದಗಳಿವೆ. ಸಾಮಾನ್ಯವಾಗಿ, ಪಠ್ಯದಲ್ಲಿ ಅಪರೂಪವಾಗಿ ಕಾಣುವ ಪದಗಳು ಬೇಕಾಗುವುದಿಲ್ಲ — ಕೆಲವೇ ವಾಕ್ಯಗಳಲ್ಲಿ ಮಾತ್ರ ಅವು ಇರುತ್ತವೆ ಮತ್ತು ಮಾದರಿ ಅವುಗಳಿಂದ ಕಲಿಯುವುದಿಲ್ಲ. ಆದ್ದರಿಂದ, ವೆಕ್ಟರೈಜರ್ ನಿರ್ಮಾಪಕಕ್ಕೆ ಒಂದು ಆರ್ಗ್ಯುಮೆಂಟ್ ನೀಡುವ ಮೂಲಕ ಶಬ್ದಕೋಶದ ಗಾತ್ರವನ್ನು ಕಡಿಮೆ ಸಂಖ್ಯೆಗೆ ಮಿತಿಗೊಳಿಸುವುದು ಅರ್ಥಪೂರ್ಣ.

ಈ ಎರಡೂ ಹಂತಗಳನ್ನು **TextVectorization** ಲೇಯರ್ ಬಳಸಿ ನಿರ್ವಹಿಸಬಹುದು. ಈಗ ನಾವು ವೆಕ್ಟರೈಜರ್ ವಸ್ತುವನ್ನು ಸೃಷ್ಟಿಸಿ, ನಂತರ ಎಲ್ಲಾ ಪಠ್ಯವನ್ನು ಪರಿಶೀಲಿಸಿ ಶಬ್ದಕೋಶವನ್ನು ನಿರ್ಮಿಸಲು `adapt` ವಿಧಾನವನ್ನು ಕರೆಸೋಣ:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **ಗಮನಿಸಿ** ನಾವು ಸಂಪೂರ್ಣ ಡೇಟಾಸೆಟ್‌ನ ಒಂದು ಉಪಸಮೂಹವನ್ನು ಮಾತ್ರ ಶಬ್ದಕೋಶವನ್ನು ನಿರ್ಮಿಸಲು ಬಳಸುತ್ತಿದ್ದೇವೆ. ನಾವು ಇದನ್ನು ಕಾರ್ಯಾಚರಣೆಯ ಸಮಯವನ್ನು ವೇಗಗೊಳಿಸಲು ಮತ್ತು ನಿಮ್ಮನ್ನು ಕಾಯದಿರಿಸಲು ಮಾಡುತ್ತೇವೆ. ಆದಾಗ್ಯೂ, ಸಂಪೂರ್ಣ ಡೇಟಾಸೆಟ್‌ನ ಕೆಲವು ಪದಗಳು ಶಬ್ದಕೋಶದಲ್ಲಿ ಸೇರಿಸಲಾಗದಿರುವ ಸಾಧ್ಯತೆ ಇದೆ ಮತ್ತು ತರಬೇತಿಯ ಸಮಯದಲ್ಲಿ ಅವುಗಳನ್ನು ನಿರ್ಲಕ್ಷಿಸಲಾಗುತ್ತದೆ. ಆದ್ದರಿಂದ, ಸಂಪೂರ್ಣ ಶಬ್ದಕೋಶದ ಗಾತ್ರವನ್ನು ಬಳಸಿಕೊಂಡು `adapt` ಸಮಯದಲ್ಲಿ ಸಂಪೂರ್ಣ ಡೇಟಾಸೆಟ್ ಮೂಲಕ ಓಡುವುದು ಅಂತಿಮ ನಿಖರತೆಯನ್ನು ಹೆಚ್ಚಿಸಬಹುದು, ಆದರೆ ಬಹಳಷ್ಟು ಅಲ್ಲ.

ಈಗ ನಾವು ನಿಜವಾದ ಶಬ್ದಕೋಶವನ್ನು ಪ್ರವೇಶಿಸಬಹುದು:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


ವೆಕ್ಟರೈಸರ್ ಬಳಸಿ, ನಾವು ಯಾವುದೇ ಪಠ್ಯವನ್ನು ಸುಲಭವಾಗಿ ಸಂಖ್ಯೆಗಳ ಸಮೂಹವಾಗಿ ಎನ್‌ಕೋಡ್ ಮಾಡಬಹುದು:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## ಪದಗಳ ಬ್ಯಾಗ್ ಪಠ್ಯ ಪ್ರತಿನಿಧಾನ

ಪದಗಳು ಅರ್ಥವನ್ನು ಪ್ರತಿನಿಧಿಸುವುದರಿಂದ, ಕೆಲವೊಮ್ಮೆ ನಾವು ವಾಕ್ಯದ ಕ್ರಮವನ್ನು ಪರಿಗಣಿಸದೆ, ಪ್ರತ್ಯೇಕ ಪದಗಳನ್ನು ನೋಡಿಕೊಂಡು ಪಠ್ಯದ ಅರ್ಥವನ್ನು ತಿಳಿದುಕೊಳ್ಳಬಹುದು. ಉದಾಹರಣೆಗೆ, ಸುದ್ದಿಗಳನ್ನು ವರ್ಗೀಕರಿಸುವಾಗ, *ಹವಾಮಾನ* ಮತ್ತು *ಹಿಮ* ಎಂಬ ಪದಗಳು *ಹವಾಮಾನ ಮುನ್ಸೂಚನೆ* ಅನ್ನು ಸೂಚಿಸುವ ಸಾಧ್ಯತೆ ಇರುತ್ತದೆ, ಆದರೆ *ಸ್ಟಾಕ್ಸ್* ಮತ್ತು *ಡಾಲರ್* ಎಂಬ ಪದಗಳು *ಆರ್ಥಿಕ ಸುದ್ದಿ* ಗೆ ಸೇರಬಹುದು.

**ಪದಗಳ ಬ್ಯಾಗ್** (BoW) ವೆಕ್ಟರ್ ಪ್ರತಿನಿಧಾನವು ಅತಿ ಸರಳವಾಗಿ ಅರ್ಥಮಾಡಿಕೊಳ್ಳಬಹುದಾದ ಪರಂಪರাগত ವೆಕ್ಟರ್ ಪ್ರತಿನಿಧಾನವಾಗಿದೆ. ಪ್ರತಿ ಪದವು ಒಂದು ವೆಕ್ಟರ್ ಸೂಚ್ಯಂಕಕ್ಕೆ ಜೋಡಿಸಲಾಗುತ್ತದೆ, ಮತ್ತು ಒಂದು ವೆಕ್ಟರ್ ಅಂಶವು ನೀಡಲಾದ ದಾಖಲೆಗಳಲ್ಲಿ ಪ್ರತಿ ಪದದ ಸಂಭವನಗಳ ಸಂಖ್ಯೆಯನ್ನು ಹೊಂದಿರುತ್ತದೆ.

![ಪದಗಳ ಬ್ಯಾಗ್ ವೆಕ್ಟರ್ ಪ್ರತಿನಿಧಾನವನ್ನು ಮೆಮೊರಿಯಲ್ಲಿ ಹೇಗೆ ಪ್ರತಿನಿಧಿಸಲಾಗುತ್ತದೆ ಎಂಬುದನ್ನು ತೋರಿಸುವ ಚಿತ್ರ.](../../../../../translated_images/kn/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: BoW ಅನ್ನು ಪಠ್ಯದ ಪ್ರತ್ಯೇಕ ಪದಗಳ ಒನ್-ಹಾಟ್-ಎನ್‌ಕೋಡ್ ಮಾಡಿದ ವೆಕ್ಟರ್‌ಗಳ ಮೊತ್ತವೆಂದು ಕೂಡ ಪರಿಗಣಿಸಬಹುದು.

ಕೆಳಗಿನ ಉದಾಹರಣೆಯಲ್ಲಿ Scikit Learn ಪೈಥಾನ್ ಲೈಬ್ರರಿಯನ್ನು ಬಳಸಿ ಪದಗಳ ಬ್ಯಾಗ್ ಪ್ರತಿನಿಧಾನವನ್ನು ಹೇಗೆ ರಚಿಸುವುದು ಎಂಬುದನ್ನು ತೋರಿಸಲಾಗಿದೆ:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

ನಾವು ಮೇಲ್ನೋಟದಲ್ಲಿ ವ್ಯಾಖ್ಯಾನಿಸಿದ ಕೇರಾಸ್ ವೆಕ್ಟರೈಜರ್ ಅನ್ನು ಕೂಡ ಬಳಸಬಹುದು, ಪ್ರತಿ ಪದ ಸಂಖ್ಯೆಯನ್ನು ಒನ್-ಹಾಟ್ ಎನ್‌ಕೋಡಿಂಗ್‌ಗೆ ಪರಿವರ್ತಿಸಿ ಮತ್ತು ಆ ಎಲ್ಲಾ ವೆಕ್ಟರ್‌ಗಳನ್ನು ಸೇರಿಸುವ ಮೂಲಕ:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **ಗಮನಿಸಿ**: ನೀವು ಹಿಂದಿನ ಉದಾಹರಣೆಯಿಂದ ಫಲಿತಾಂಶ ಭಿನ್ನವಾಗಿರುವುದನ್ನು ಆಶ್ಚರ್ಯಪಡಬಹುದು. ಕಾರಣವೆಂದರೆ Keras ಉದಾಹರಣೆಯಲ್ಲಿ ವೆಕ್ಟರ್‌ನ ಉದ್ದವು ಶಬ್ದಕೋಶದ ಗಾತ್ರಕ್ಕೆ ಹೊಂದಿಕೆಯಾಗಿದ್ದು, ಅದು ಸಂಪೂರ್ಣ AG News ಡೇಟಾಸೆಟ್‌ನಿಂದ ನಿರ್ಮಿಸಲ್ಪಟ್ಟಿದೆ, ಆದರೆ Scikit Learn ಉದಾಹರಣೆಯಲ್ಲಿ ನಾವು ಶಬ್ದಕೋಶವನ್ನು ಉದಾಹರಣೆಯ ಪಠ್ಯದಿಂದ ತಕ್ಷಣ ನಿರ್ಮಿಸಿದ್ದೇವೆ.


## BoW ವರ್ಗೀಕರಣೆಯನ್ನು ತರಬೇತುಗೊಳಿಸುವುದು

ನಾವು ಈಗ ನಮ್ಮ ಪಠ್ಯದ ಬ್ಯಾಗ್-ಆಫ್-ವರ್ಡ್ಸ್ ಪ್ರತಿನಿಧಾನವನ್ನು ಹೇಗೆ ನಿರ್ಮಿಸುವುದನ್ನು ಕಲಿತಿದ್ದೇವೆ, ಅದನ್ನು ಬಳಸುವ ವರ್ಗೀಕರಣೆಯನ್ನು ತರಬೇತುಗೊಳಿಸೋಣ. ಮೊದಲು, ನಮ್ಮ ಡೇಟಾಸೆಟ್ ಅನ್ನು ಬ್ಯಾಗ್-ಆಫ್-ವರ್ಡ್ಸ್ ಪ್ರತಿನಿಧಾನಕ್ಕೆ ಪರಿವರ್ತಿಸಬೇಕಾಗುತ್ತದೆ. ಇದನ್ನು ಕೆಳಗಿನಂತೆ `map` ಫಂಕ್ಷನ್ ಬಳಸಿ ಸಾಧಿಸಬಹುದು:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

ಈಗ ನಾವು ಒಂದು ಸರಳ ವರ್ಗೀಕರಣ ನ್ಯೂರಲ್ ನೆಟ್‌ವರ್ಕ್ ಅನ್ನು ವ್ಯಾಖ್ಯಾನಿಸೋಣ, ಇದರಲ್ಲಿ ಒಂದು ಲೀನಿಯರ್ ಲೇಯರ್ ಇದೆ. ಇನ್‌ಪುಟ್ ಗಾತ್ರವು `vocab_size` ಆಗಿದ್ದು, ಔಟ್‌ಪುಟ್ ಗಾತ್ರವು ವರ್ಗಗಳ ಸಂಖ್ಯೆಗೆ (4) ಹೊಂದಿಕೆಯಾಗುತ್ತದೆ. ನಾವು ವರ್ಗೀಕರಣ ಕಾರ್ಯವನ್ನು ಪರಿಹರಿಸುತ್ತಿದ್ದೇವೆ ಆದ್ದರಿಂದ, ಅಂತಿಮ ಸಕ್ರಿಯತೆ ಕಾರ್ಯ **softmax** ಆಗಿದೆ:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


ನಾವು 4 ವರ್ಗಗಳನ್ನು ಹೊಂದಿರುವುದರಿಂದ, 80% ಕ್ಕಿಂತ ಹೆಚ್ಚು ಶುದ್ಧತೆ ಒಳ್ಳೆಯ ಫಲಿತಾಂಶವಾಗಿದೆ.

## ಒಂದು ನೆಟ್‌ವರ್ಕ್ ಆಗಿ ವರ್ಗೀಕರಿಸುವ ಯಂತ್ರವನ್ನು ತರಬೇತುಗೊಳಿಸುವುದು

ವ್ಯಕ್ತಿಗತಕಾರಕವೂ Keras ಲೇಯರ್ ಆಗಿರುವುದರಿಂದ, ಅದನ್ನು ಒಳಗೊಂಡಿರುವ ನೆಟ್‌ವರ್ಕ್ ಅನ್ನು ನಾವು ವ್ಯಾಖ್ಯಾನಿಸಿ, ಅಂತ್ಯದಿಂದ ಅಂತ್ಯಕ್ಕೆ ತರಬೇತುಗೊಳಿಸಬಹುದು. ಈ ರೀತಿಯಲ್ಲಿ ನಾವು `map` ಬಳಸಿ ಡೇಟಾಸೆಟ್ ಅನ್ನು ವ್ಯಕ್ತಿಗತಗೊಳಿಸುವ ಅಗತ್ಯವಿಲ್ಲ, ನಾವು ಮೂಲ ಡೇಟಾಸೆಟ್ ಅನ್ನು ನೇರವಾಗಿ ನೆಟ್‌ವರ್ಕ್‌ನ ಇನ್‌ಪುಟ್‌ಗೆ ನೀಡಬಹುದು.

> **Note**: ನಮಗೆ ಇನ್ನೂ ಡಿಕ್ಷನರಿಗಳಿಂದ (ಹಾಗು `title`, `description` ಮತ್ತು `label` ಮುಂತಾದ) ಕ್ಷೇತ್ರಗಳನ್ನು ಟ್ಯೂಪಲ್‌ಗಳಿಗೆ ಪರಿವರ್ತಿಸಲು ನಮ್ಮ ಡೇಟಾಸೆಟ್‌ಗೆ ಮ್ಯಾಪ್‌ಗಳನ್ನು ಅನ್ವಯಿಸಬೇಕಾಗುತ್ತದೆ. ಆದಾಗ್ಯೂ, ಡಿಸ್ಕ್‌ನಿಂದ ಡೇಟಾವನ್ನು ಲೋಡ್ ಮಾಡುವಾಗ, ನಾವು ಮೊದಲಿನಿಂದಲೇ ಅಗತ್ಯವಿರುವ ರಚನೆಯೊಂದಿಗೆ ಡೇಟಾಸೆಟ್ ಅನ್ನು ನಿರ್ಮಿಸಬಹುದು.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## ಬಿಗ್ರಾಮ್, ಟ್ರಿಗ್ರಾಮ್ ಮತ್ತು ಎನ್-ಗ್ರಾಮ್‌ಗಳು

ಬ್ಯಾಗ್-ಆಫ್-ವರ್ಡ್ಸ್ ವಿಧಾನದ ಒಂದು ಮಿತಿ ಎಂದರೆ ಕೆಲವು ಪದಗಳು ಬಹುಪದ ಅಭಿವ್ಯಕ್ತಿಗಳ ಭಾಗವಾಗಿರುತ್ತವೆ, ಉದಾಹರಣೆಗೆ, 'ಹಾಟ್ ಡಾಗ್' ಎಂಬ ಪದವು 'ಹಾಟ್' ಮತ್ತು 'ಡಾಗ್' ಎಂಬ ಪದಗಳಿಗಿಂತ ಸಂಪೂರ್ಣ ವಿಭಿನ್ನ ಅರ್ಥ ಹೊಂದಿದೆ. ನಾವು 'ಹಾಟ್' ಮತ್ತು 'ಡಾಗ್' ಪದಗಳನ್ನು ಯಾವಾಗಲೂ ಒಂದೇ ವೆಕ್ಟರ್‌ಗಳ ಮೂಲಕ ಪ್ರತಿನಿಧಿಸಿದರೆ, ಅದು ನಮ್ಮ ಮಾದರಿಯನ್ನು ಗೊಂದಲಕ್ಕೆ ಒಳಪಡಿಸಬಹುದು.

ಇದನ್ನು ಪರಿಹರಿಸಲು, **ಎನ್-ಗ್ರಾಮ್ ಪ್ರತಿನಿಧಾನಗಳು** ಡಾಕ್ಯುಮೆಂಟ್ ವರ್ಗೀಕರಣ ವಿಧಾನಗಳಲ್ಲಿ ಸಾಮಾನ್ಯವಾಗಿ ಬಳಸಲಾಗುತ್ತವೆ, ಇಲ್ಲಿ ಪ್ರತಿ ಪದ, ದ್ವಿಪದ ಅಥವಾ ತ್ರಿಪದದ ಆವರ್ತನೆ ತರಬೇತಿ ವರ್ಗೀಕರಿಸುವವರಿಗೆ ಉಪಯುಕ್ತ ಲಕ್ಷಣವಾಗಿರುತ್ತದೆ. ಉದಾಹರಣೆಗೆ, ಬಿಗ್ರಾಮ್ ಪ್ರತಿನಿಧಾನಗಳಲ್ಲಿ, ಮೂಲ ಪದಗಳ ಜೊತೆಗೆ ಎಲ್ಲಾ ಪದ ಜೋಡಿಗಳನ್ನು ಶಬ್ದಕೋಶಕ್ಕೆ ಸೇರಿಸಲಾಗುತ್ತದೆ.

ಕೆಳಗಿನ ಉದಾಹರಣೆಯಲ್ಲಿ Scikit Learn ಬಳಸಿ ಬಿಗ್ರಾಮ್ ಬ್ಯಾಗ್ ಆಫ್ ವರ್ಡ್ ಪ್ರತಿನಿಧಾನವನ್ನು ಹೇಗೆ ರಚಿಸುವುದು ಎಂಬುದನ್ನು ತೋರಿಸಲಾಗಿದೆ:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

n-ಗ್ರಾಮ್ ವಿಧಾನದ ಪ್ರಮುಖ ದೋಷವೆಂದರೆ ಶಬ್ದಕೋಶದ ಗಾತ್ರ ಅತ್ಯಂತ ವೇಗವಾಗಿ ಹೆಚ್ಚಾಗುತ್ತದೆ. ಪ್ರಾಯೋಗಿಕವಾಗಿ, ನಾವು n-ಗ್ರಾಮ್ ಪ್ರತಿನಿಧಾನವನ್ನು *ಎಂಬೆಡ್ಡಿಂಗ್‌ಗಳು* ಎಂಬ ಆಯಾಮ ಕಡಿತ ತಂತ್ರಜ್ಞಾನದೊಂದಿಗೆ ಸಂಯೋಜಿಸಬೇಕಾಗುತ್ತದೆ, ಇದನ್ನು ನಾವು ಮುಂದಿನ ಘಟಕದಲ್ಲಿ ಚರ್ಚಿಸುವೆವು.

ನಮ್ಮ **AG News** ಡೇಟಾಸೆಟ್‌ನಲ್ಲಿ n-ಗ್ರಾಮ್ ಪ್ರತಿನಿಧಾನವನ್ನು ಬಳಸಲು, ನಾವು `TextVectorization` ನಿರ್ಮಾಪಕಕ್ಕೆ `ngrams` ಪರಿಮಾಣವನ್ನು ಪಾಸ್ ಮಾಡಬೇಕಾಗುತ್ತದೆ. ಬಿಗ್ರಾಮ್ ಶಬ್ದಕೋಶದ ಉದ್ದವು **ಗಣನೀಯವಾಗಿ ದೊಡ್ಡದು**, ನಮ್ಮ ಪ್ರಕರಣದಲ್ಲಿ ಇದು 1.3 ಮಿಲಿಯನ್ ಟೋಕನ್‌ಗಳಿಗಿಂತ ಹೆಚ್ಚು! ಆದ್ದರಿಂದ, ಬಿಗ್ರಾಮ್ ಟೋಕನ್‌ಗಳ ಸಂಖ್ಯೆಯನ್ನು ಸಹ ಯುಕ್ತಿಯುತ ಸಂಖ್ಯೆಯಿಂದ ಮಿತಿಗೊಳಿಸುವುದು ಸೂಕ್ತ.

ಮೇಲಿನ ಕೋಡ್ ಅನ್ನು ಬಳಸಿ ವರ್ಗೀಕರಣಕಾರಿಯನ್ನು ತರಬೇತುಗೊಳಿಸಬಹುದು, ಆದರೆ ಅದು ಬಹಳಷ್ಟು ಮೆಮೊರಿ ಅಸಮರ್ಥವಾಗಿರುತ್ತದೆ. ಮುಂದಿನ ಘಟಕದಲ್ಲಿ, ನಾವು ಎಂಬೆಡ್ಡಿಂಗ್‌ಗಳನ್ನು ಬಳಸಿ ಬಿಗ್ರಾಮ್ ವರ್ಗೀಕರಣಕಾರಿಯನ್ನು ತರಬೇತುಗೊಳಿಸುವೆವು. ಈ ನಡುವೆ, ನೀವು ಈ ನೋಟ್ಬುಕ್‌ನಲ್ಲಿ ಬಿಗ್ರಾಮ್ ವರ್ಗೀಕರಣಕಾರಿಯ ತರಬೇತಿಯನ್ನು ಪ್ರಯೋಗಿಸಿ, ಹೆಚ್ಚಿನ ನಿಖರತೆಯನ್ನು ಪಡೆಯಬಹುದೇ ಎಂದು ನೋಡಬಹುದು.


## ಸ್ವಯಂಚಾಲಿತವಾಗಿ BoW ವೆಕ್ಟರ್‌ಗಳನ್ನು ಲೆಕ್ಕಹಾಕುವುದು

ಮೇಲಿನ ಉದಾಹರಣೆಯಲ್ಲಿ ನಾವು ಪ್ರತ್ಯೇಕ ಪದಗಳ ಒನ್-ಹಾಟ್ ಎನ್‌ಕೋಡಿಂಗ್‌ಗಳನ್ನು ಸೇರಿಸುವ ಮೂಲಕ ಕೈಯಿಂದ BoW ವೆಕ್ಟರ್‌ಗಳನ್ನು ಲೆಕ್ಕಹಾಕಿದ್ದೇವೆ. ಆದರೆ, TensorFlow ನ ಇತ್ತೀಚಿನ ಆವೃತ್ತಿ `output_mode='count'` ಪರಾಮೀಟರ್ ಅನ್ನು ವೆಕ್ಟರೈಜರ್ ನಿರ್ಮಾಪಕಕ್ಕೆ ನೀಡುವ ಮೂಲಕ BoW ವೆಕ್ಟರ್‌ಗಳನ್ನು ಸ್ವಯಂಚಾಲಿತವಾಗಿ ಲೆಕ್ಕಹಾಕಲು ಅನುಮತಿಸುತ್ತದೆ. ಇದರಿಂದ ನಮ್ಮ ಮಾದರಿಯನ್ನು ವ್ಯಾಖ್ಯಾನಿಸುವುದು ಮತ್ತು ತರಬೇತುಗೊಳಿಸುವುದು ಬಹಳ ಸುಲಭವಾಗುತ್ತದೆ:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## ಪದದ ಆವರ್ತನೆ - ಪ್ರತಿಕೂಲ ದಾಖಲೆ ಆವರ್ತನೆ (TF-IDF)

BoW ಪ್ರತಿನಿಧಾನದಲ್ಲಿ, ಪದಗಳ ಸಂಭವಗಳನ್ನು ಪದದ ಸ್ವಭಾವವನ್ನು ಪರಿಗಣಿಸದೆ ಒಂದೇ ತಂತ್ರವನ್ನು ಬಳಸಿ ತೂಕ ನೀಡಲಾಗುತ್ತದೆ. ಆದರೆ, *a* ಮತ್ತು *in* ಹೀಗೆ ಸಾಮಾನ್ಯವಾಗಿ ಬರುತ್ತಿರುವ ಪದಗಳು ವಿಶೇಷ ಪದಗಳಿಗಿಂತ ವರ್ಗೀಕರಣಕ್ಕೆ ಕಡಿಮೆ ಮಹತ್ವದವು ಎಂಬುದು ಸ್ಪಷ್ಟವಾಗಿದೆ. ಬಹುತೇಕ NLP ಕಾರ್ಯಗಳಲ್ಲಿ ಕೆಲವು ಪದಗಳು ಇತರರಿಗಿಂತ ಹೆಚ್ಚು ಪ್ರಾಸಂಗಿಕವಾಗಿರುತ್ತವೆ.

**TF-IDF** ಎಂದರೆ **term frequency - inverse document frequency**. ಇದು bag-of-words ನ ಒಂದು ಬದಲಾವಣೆ, ಇಲ್ಲಿ ಒಂದು ದಾಖಲೆದಲ್ಲಿ ಪದದ ಕಾಣಿಕೆ 0/1 ಬೈನರಿ ಮೌಲ್ಯ ಬದಲಾಗಿ, ಪದದ ಸಂಭವದ ಆವರ್ತನೆಗೆ ಸಂಬಂಧಿಸಿದ ಫ್ಲೋಟಿಂಗ್-ಪಾಯಿಂಟ್ ಮೌಲ್ಯವನ್ನು ಬಳಸಲಾಗುತ್ತದೆ.

ವೈಜ್ಞಾನಿಕವಾಗಿ, ದಾಖಲೆ $j$ ಯಲ್ಲಿನ ಪದ $i$ ಯ ತೂಕ $w_{ij}$ ಅನ್ನು ಹೀಗೆ ವ್ಯಾಖ್ಯಾನಿಸಲಾಗುತ್ತದೆ:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
ಇಲ್ಲಿ
* $tf_{ij}$ ಎಂದರೆ $j$ ನಲ್ಲಿ $i$ ಪದದ ಸಂಭವಗಳ ಸಂಖ್ಯೆ, ಅಂದರೆ ನಾವು ಹಿಂದಿನ BoW ಮೌಲ್ಯವನ್ನು ನೋಡಿದ್ದೇವೆ
* $N$ ಎಂದರೆ ಸಂಗ್ರಹದಲ್ಲಿನ ದಾಖಲೆಗಳ ಸಂಖ್ಯೆ
* $df_i$ ಎಂದರೆ ಸಂಪೂರ್ಣ ಸಂಗ್ರಹದಲ್ಲಿ ಪದ $i$ ಹೊಂದಿರುವ ದಾಖಲೆಗಳ ಸಂಖ್ಯೆ

TF-IDF ಮೌಲ್ಯ $w_{ij}$ ಒಂದು ಪದವು ದಾಖಲೆಗಳಲ್ಲಿ ಎಷ್ಟು ಬಾರಿ ಬರುತ್ತದೆ ಎಂಬುದರ ಅನುಪಾತದಲ್ಲಿ ಹೆಚ್ಚಾಗುತ್ತದೆ ಮತ್ತು ಆ ಪದವನ್ನು ಹೊಂದಿರುವ ದಾಖಲೆಗಳ ಸಂಖ್ಯೆಯಿಂದ ಕಡಿಮೆ ಮಾಡಲಾಗುತ್ತದೆ, ಇದು ಕೆಲವು ಪದಗಳು ಇತರರಿಗಿಂತ ಹೆಚ್ಚು ಬಾರಿ ಬರುತ್ತವೆ ಎಂಬುದನ್ನು ಸರಿಹೊಂದಿಸಲು ಸಹಾಯ ಮಾಡುತ್ತದೆ. ಉದಾಹರಣೆಗೆ, ಒಂದು ಪದವು ಸಂಗ್ರಹದಲ್ಲಿನ *ಪ್ರತಿ* ದಾಖಲೆದಲ್ಲಿಯೂ ಇದ್ದರೆ, $df_i=N$, ಆಗ $w_{ij}=0$ ಆಗಿ ಆ ಪದಗಳನ್ನು ಸಂಪೂರ್ಣವಾಗಿ ನಿರ್ಲಕ್ಷಿಸಲಾಗುತ್ತದೆ.

ನೀವು ಸುಲಭವಾಗಿ Scikit Learn ಬಳಸಿ ಪಠ್ಯದ TF-IDF ವೆಕ್ಟರೈಜೆಶನ್ ಸೃಷ್ಟಿಸಬಹುದು:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

Keras ನಲ್ಲಿ, `TextVectorization` ಲೇಯರ್ ಸ್ವಯಂಚಾಲಿತವಾಗಿ TF-IDF ಆವರ್ತನೆಗಳನ್ನು ಲೆಕ್ಕಹಾಕಬಹುದು `output_mode='tf-idf'` ಪರಿಮಾಣವನ್ನು ನೀಡುವ ಮೂಲಕ. ಮೇಲಿನ ಕೋಡ್ ಅನ್ನು ಪುನರಾವರ್ತಿಸೋಣ ಮತ್ತು TF-IDF ಬಳಕೆ ನಿಖರತೆಯನ್ನು ಹೆಚ್ಚಿಸುವುದೇ ಎಂದು ನೋಡೋಣ:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## ಸಮಾರೋಪ

TF-IDF ಪ್ರತಿನಿಧಿಗಳು ವಿಭಿನ್ನ ಪದಗಳಿಗೆ ಆವರ್ತನ ತೂಕಗಳನ್ನು ನೀಡಿದರೂ, ಅವು ಅರ್ಥ ಅಥವಾ ಕ್ರಮವನ್ನು ಪ್ರತಿನಿಧಿಸಲು ಸಾಧ್ಯವಿಲ್ಲ. 1935 ರಲ್ಲಿ ಪ್ರಸಿದ್ಧ ಭಾಷಾಶಾಸ್ತ್ರಜ್ಞ ಜೆ. ಆರ್. ಫರ್ಥ್ ಹೇಳಿದಂತೆ, "ಒಂದು ಪದದ ಸಂಪೂರ್ಣ ಅರ್ಥ ಯಾವಾಗಲೂ ಸಾಂದರ್ಭಿಕವಾಗಿರುತ್ತದೆ, ಮತ್ತು ಸಾಂದರ್ಭಿಕತೆಯ ಹೊರಗಿನ ಅರ್ಥ ಅಧ್ಯಯನವನ್ನು ಗಂಭೀರವಾಗಿ ತೆಗೆದುಕೊಳ್ಳಲಾಗುವುದಿಲ್ಲ." ನಾವು ಈ ಕೋರ್ಸ್‌ನಲ್ಲಿ ನಂತರ ಭಾಷಾ ಮಾದರೀಕರಣವನ್ನು ಬಳಸಿ ಪಠ್ಯದಿಂದ ಸಾಂದರ್ಭಿಕ ಮಾಹಿತಿಯನ್ನು ಹೇಗೆ ಹಿಡಿಯುವುದು ಎಂಬುದನ್ನು ಕಲಿಯುತ್ತೇವೆ.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸ್ವೀಕರಣ**:  
ಈ ದಸ್ತಾವೇಜು AI ಅನುವಾದ ಸೇವೆ [Co-op Translator](https://github.com/Azure/co-op-translator) ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ನಿಖರತೆಯಿಗಾಗಿ ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ತಪ್ಪುಗಳು ಅಥವಾ ಅಸತ್ಯತೆಗಳು ಇರಬಹುದು ಎಂದು ದಯವಿಟ್ಟು ಗಮನಿಸಿ. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಸ್ತಾವೇಜನ್ನು ಅಧಿಕೃತ ಮೂಲವೆಂದು ಪರಿಗಣಿಸಬೇಕು. ಮಹತ್ವದ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪು ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವಿಕೆ ಅಥವಾ ತಪ್ಪು ವಿವರಣೆಗಳಿಗೆ ನಾವು ಹೊಣೆಗಾರರಾಗುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
